### Get Water Rights Data from WaDE API and Plot Allocation Amounts for Multiple States

This code calls the WaDE 2.0 API and plots water rights amounts for multiple states.
The objective of this code was to test the API access by specifiying State ID and concatenating the resulting data. Because of that, the data access are limited into 1000 rows for each state.

Another Jupyter notebook (plotAllocations_indexedSearch) tests obtaining the whole data for a given state by using indexed (paged) access of the API.

The code:
1. Calls the WaDE API and gets water rights table in JSON format.

2. Extracts from the JSON data for water allocations which includes information for water source, location (sites), water allocation amounts, beneficial uses, etc.

3. Organizes the data into a Pandas data frame.

4. Plosts water allocation amount in Google map and Plotly's mapbox.


#####  Required packages

    - Pandas
    - Numpy
    - JSON
    - gmaps
    - plotly

Install required packages (from command line or here) if the they have not been installed already.If running from Jupyter notebook use the cell magic: %%cmd

    %%cmd
    pip install gmaps
    
    pip install plotly


In addition, you may need to enable the following extensions:

    jupyter nbextension enable --py --sys-prefix widgetsnbextension

    jupyter nbextension enable --py --sys-prefix gmaps


In [ ]:
#!/usr/bin/env python
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
from urllib.request import urlopen
import gmaps
import gmaps.datasets
import plotly.express as px

In [ ]:
# # Access WaDE API to get the water allocations JSON 
# url = 'https://wade-api-qa.azure-api.net/v1/SiteAllocationAmounts?State=WA'

# # extract target columns
# subcolumns = ['WaterSourceUUID', 'Sites', 'AllocationAmount', 'AllocationMaximum',
#               'BeneficialUses']

# response =  urlopen(url)
# dataread = response.read().decode("utf-8")
# data = json.loads(dataread)
# data
# df10 = json_normalize(data, 'Organizations')
# df10
# df20 = df10['WaterAllocations']
# df20
# df30 = pd.concat([pd.DataFrame(json_normalize(x)) for x in df10['WaterAllocations']], 
#                  ignore_index=True)
# df100 = df30[subcolumns]

# df100.head(5)

In [ ]:
# Access WaDE API to get the water allocations JSON 
url = 'https://wade-api-qa.azure-api.net/v1/SiteAllocationAmounts?State='
statesShort = ["CO", "UT", "WA", "OR", "CA", "OK", "ND", "AZ"]

df100_list = []

# extract target columns
subcolumns = ['WaterSourceUUID', 'Sites', 'AllocationAmount', 'AllocationMaximum',
              'BeneficialUses']

for state in statesShort:
    urlwithfilter = url+state
    response =  urlopen(urlwithfilter)
    dataread = response.read().decode("utf-8")
    data = json.loads(dataread)
    df10 = json_normalize(data, 'Organizations')
    df20 = pd.concat([pd.DataFrame(json_normalize(x)) for x in df10['WaterAllocations']],
                     ignore_index=True)
    df30 = df20[subcolumns]
    df100_list.append(df30)

df100 = pd.concat(df100_list, sort=True, ignore_index=True)

#df100.drop_duplicates(inplace=True)
print(len(df100.index))

df100.head(5)

In [ ]:
# get a data frame that combines lat lon with allocation values

latloncolumns = ['WaterSourceUUID','Longitude', 'Latitude',
                 'AllocationAmount', 'AllocationMaximum', 'BeneficialUses']

df300 = pd.DataFrame(columns=latloncolumns)

jy = 0
for index, rows in df100.iterrows(): 
    SitesL = rows.Sites
    for latlon in SitesL:
        #print(latlon)
        df300.loc[jy,'WaterSourceUUID'] = rows.WaterSourceUUID
        df300.loc[jy,'AllocationAmount'] = rows.AllocationAmount
        df300.loc[jy,'AllocationMaximum'] = rows.AllocationMaximum
        df300.loc[jy,'BeneficialUses'] = rows.BeneficialUses

        df300.loc[jy,'Longitude'] = latlon['Longitude']
        df300.loc[jy,'Latitude'] = latlon['Latitude']
        jy += 1

print(len(df300.index))
df300.head(5)

# outdf100.WaterSourceUUID = df100['WaterSourceUUID']

In [ ]:
print("Drop rows without lat lon values...")

df500 = df300.dropna(subset=['Longitude', 'Latitude'])
df500 = df500.reset_index(drop=True)

print(len(df500.index))
df500.head(5)

In [ ]:
print("Drop duplicates if there are any...")

subCols = ['Longitude', 'Latitude']

df500.drop_duplicates(subset = subCols, inplace=True)   #
df500 = df500.reset_index(drop=True)

print(len(df500.index))
df500.head(5)

In [ ]:
# make sure the data are in the right data types
# plotly complained about allocation types being 'object'

print(df500.dtypes)

df500['AllocationAmount'] = pd.to_numeric(df500['AllocationAmount'], errors='coerce')
df500['AllocationMaximum'] = pd.to_numeric(df500['AllocationMaximum'], errors='coerce')
df500['Latitude'] = pd.to_numeric(df500['Latitude'], errors='coerce')
df500['Longitude'] = pd.to_numeric(df500['Longitude'], errors='coerce')
print(df500.dtypes)

###### Make sure to get API keys from Google and Mapbox

In [ ]:
# Plot allocation amount as a gmaps heatmap

APIKey = 'AI.......' # put your Google API key here
gmaps.configure(api_key=APIKey)

logan_coordinates = (41.6, -111.8)
denver_coordinates = (39.78, -104.59)
fig = gmaps.figure(map_type='HYBRID', center=denver_coordinates, zoom_level=4.5)

locations = df500[['Latitude', 'Longitude']]
#locations = locations[0:8701]
weights = df500['AllocationAmount']
#weights = weights1[0:8701]
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))

fig

###### If using the token file, make sure to put a mapbox token file (.mapbox_token) inside the directory of this source code

In [ ]:
print("Droping null amounts...")

df500purge = df500.loc[(df500["AllocationAmount"] == '') | (df500["AllocationAmount"] == np.nan)]
if len(df500purge.index) > 0:
    dropIndex = df500.loc[(df500["AllocationAmount"] == '') | (df500["AllocationAmount"] == np.nan)].index
    outdf100 = df500.drop(dropIndex)
    outdf100 = df500.reset_index(drop=True)

In [ ]:
print("Droping null max amounts...")

df500purge = df500.loc[(df500["AllocationMaximum"] == '') | (df500["AllocationMaximum"] == np.nan)]
if len(df500purge.index) > 0:
    dropIndex = df500.loc[(df500["AllocationMaximum"] == '') | (df500["AllocationMaximum"] == np.nan)].index
    outdf100 = df500.drop(dropIndex)
    outdf100 = df500.reset_index(drop=True)

In [ ]:
# plot allocation amount as plotly heatmap

#need to save your mapbox token file in the same dir
px.set_mapbox_access_token(open(".mapbox_token").read())

fig = px.scatter_mapbox(df500, lat="Latitude", lon="Longitude",  
                        color="AllocationAmount", #size="AllocationMaximum",
                  color_continuous_scale=px.colors.cyclical.IceFire, size_max=5,
                        range_color=[0,1000],zoom=3, hover_data=["BeneficialUses"])
fig.show()